# <font color='red'>Modelo de Classificação - Machine Learning</font>

## <font color='blue'>Gabriel Lima Gomes</font>

http://scikit-learn.org/stable/modules/naive_bayes.html

## Multinomial Naive Bayes - Scikit-Learn

O classificador Multinomial Naive Bayes é adequado para classificação com variáveis discretas (por exemplo, contagens de palavras para a classificação de texto). 
Pode ser utilizada term frequency ou term frequency invert document frenquency.
Documentação:

http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html

### Classificador de Textos (emails)

http://qwone.com/~jason/20Newsgroups/

In [1]:
## IMPORTANDO LIBS QUE SERÃO UTILIZADAS
import numpy as np
import pandas as pd
#from pprint import pprint

#from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

In [2]:
## DEFININDO AS CLASSES(RÓTULOS) QUE SERÃO UTILIZADAS
# (usando apenas 4 de um total de 20 disponível para que o processo de classificação seja mais rápido)
#categories = ['rec.autos', 'soc.religion.christian', 'comp.graphics', 'sci.med']

# EXTRAINDO DADOS PARA TREINAMENTO DA FONTE DE ORIGEM
#treino = fetch_20newsgroups(subset = 'train', categories = categories, shuffle = True, random_state = 42)

#df = pd.DataFrame(data= {'text':treino.data[:],'target': treino.target[:],'target_name': np.array(treino.target_names)[treino.target[:]]})
#df.to_csv("email_treino.csv",index=False)

In [4]:
"""teste = fetch_20newsgroups(subset = 'test', categories = categories, shuffle = True, random_state = 42)

df = pd.DataFrame(data= {'text':teste.data[:],
                         'target': teste.target[:],
                         'target_name': np.array(teste.target_names)[teste.target[:]]
                        })
df.to_csv("email_teste.csv",index=False)
"""

'teste = fetch_20newsgroups(subset = \'test\', categories = categories, shuffle = True, random_state = 42)\n\ndf = pd.DataFrame(data= {\'text\':teste.data[:],\n                         \'target\': teste.target[:],\n                         \'target_name\': np.array(teste.target_names)[teste.target[:]]\n                        })\ndf.to_csv("email_teste.csv",index=False)\n'

In [3]:
## CARREGANDO DADOS DE TREINO DO ARQUIVO CSV
treino = pd.read_csv("email_treino.csv")

In [4]:
## VISUALIZANDO AS PRIMEIRAS LINHAS DA TABELA
treino.head()

,target,target_name,text
0,1,rec.autos,From: jimf@centerline.com (Jim Frost)\r\nSubje...
1,1,rec.autos,From: cs173sbw@sdcc5.ucsd.edu (cs173sbw)\r\nSu...
2,1,rec.autos,From: qazi@csd4.csd.uwm.edu (Aamir Hafeez Qazi...
3,1,rec.autos,"From: ""tom neumann"" <tom.neumann@canrem.com>\r..."
4,3,soc.religion.christian,From: Rick_Granberry@pts.mot.com (Rick Granber...


In [5]:
## EXTRAINDO TARGETS DA BASE DE DADOS (VALORES ÚNICOS)
np.unique(treino.target_name)

array(['comp.graphics', 'rec.autos', 'sci.med', 'soc.religion.christian'], dtype=object)

### Bag of Words (Saco de Palavras)

In [6]:
## Tokenizing - cria um objeto com todos os documentos(linhas) e palavras separadas (colunas)
count_vect     = CountVectorizer()
X_train_counts = count_vect.fit_transform(treino.text)

## VERIFICANDO AS DIMENSÕES DO OBJETO(LINHAS x COLUNAS)
X_train_counts.shape

(2371, 36764)

In [7]:
## CRIANDO UMA tf-idf - Term Frequency times Inverse Document Frequency
tfidf_transformer = TfidfTransformer().fit(X_train_counts)
X_train_tfidf = tfidf_transformer.transform(X_train_counts)
X_train_tfidf.shape

(2371, 36764)

In [ ]:
# Mesmo resultado da célula anterior, mas combinando as funções
#tfidf_transformer = TfidfTransformer()
#X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
#X_train_tfidf.shape

In [8]:
# Criando o modelo Multinomial com os dados de treino criados no tf-idf anteriormente e as classes/rótulos de cada um
clf = MultinomialNB().fit(X_train_tfidf, treino.target)

In [9]:
# Previsões
docs_new = ['God is love', 'I use OpenGL on the GPU', 'My car is fast','doctors and nurses are good in the hospital ']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
    print('%r => %s (%i) ' % (doc, np.unique(treino.target_name)[category], category) )

'God is love' => soc.religion.christian (3) 
'I use OpenGL on the GPU' => comp.graphics (0) 
'My car is fast' => rec.autos (1) 
'doctors and nurses are good in the hospital ' => sci.med (2) 


In [10]:
## TESTANDO ACURÁCIA DO ALGORITMO DE CLASSIFICAÇÃO
teste = pd.read_csv("email_teste.csv")
docs_test = teste.text

X_new_counts = count_vect.transform(docs_test)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

## MÉDIA DE ACERTOS (classes/rótulos previstos x classes originais)
np.mean(predicted == teste.target)


0.9132362254591514

### Fim